<a href="https://colab.research.google.com/github/HerlanAssis/P-median-problem/blob/main/P_Median_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problema das *p-medianas*

"A localização de p-medianas é um problema clássico de otimização combinatória. O objetivo é localizar em uma rede p nós (denominados medianas), de forma a minimizar a soma das distâncias de cada nó de demanda até sua mediana mais próxima" (Senne e Lorena, 2003). 


Segundo (Senne e Lorena, 2003), o problema de p-medianas pode ser modelado como o seguinte problema (P) de programação inteira binária:

![](https://www.scielo.br/img/revistas/prod/v13n3/a06eq01.gif)

sujeito a:

![](https://www.scielo.br/img/revistas/prod/v13n3/a06eq02.gif)


[Texto completo: Abordagens complementares para problemas de p-medianas](https://www.scielo.br/scielo.php?pid=S0103-65132003000300007&script=sci_arttext&tlng=pt)

### Instalando dependências

Para executarmos um exemplo do problema das *p-medianas* precisamos das seguintes dependências:
* ibmdecisionoptimization
* cplex
* docplex

Para fazer isso vamos utilizar o ambiente virtual *anaconda*, como ele conseguiremos configurar um abiente utilizando um notabook python.

*OBS: Para configurar o ambiente com o conda utilizei o seguinte link [Stackoverflow: install conda package to google colab](https://stackoverflow.com/questions/59330876/install-conda-package-to-google-colab)*

In [1]:
'''
There are 2 problems that must be solved:

ujson will normally upgrade to python 3.7, must avoid this.
path to conda library is changed, must update it.
For 1, you need to add python=3.6 to conda install.

For 2, you need to add path to /usr/local/lib/python3.6/site-packages

Here's the new code
'''

# same
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
# update 1
!conda install -q -y --prefix /usr/local python=3.6 ujson
# update 2
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

--2020-11-09 19:28:50--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...

In [2]:
!conda install -c ibmdecisionoptimization cplex docplex --yes
from docplex.cp.model import CpoModel

Solving environment: - \ | / - \ | done

# All requested packages already installed.



### Instâncias de exemplo

In [3]:
'''
p: número de facilidades a serem escolhidas
m: clientes
n: numeros de medianas possíveis
d: matriz de distâncias do cliente para facilidade
'''
m = 5
n = 8
p = 3
d = [[25.0, 21.67, 21.67, 13.33, 8.33, 18.33, 8.33, 21.67],
     [6.67, 16.67, 11.67, 20.0, 15.0, 15.0, 8.33, 15.0],
     [6.67, 16.67, 15.0, 13.33, 13.33, 16.67, 21.67, 18.33],
     [25.0, 8.33, 11.67, 23.33, 16.67, 21.67, 20.0, 18.33],
     [18.33, 21.67, 13.33, 6.67, 21.67, 23.33, 18.33, 8.33]]

### Código Principal

In [4]:
clients = range(m)
places = range(n)

mdl = CpoModel()

x = [[mdl.integer_var(min=0, max=1, name="X[{}][{}] {}".format(place, client, d[client][place])) for place in places] for client in clients]

y = [mdl.integer_var(min=0, max=1, name="Y[{}] {}".format(place, place)) for place in places]

# Função objetivo
fo = mdl.sum(d[i][j]*x[i][j] for i in clients for j in places)
mdl.add(mdl.minimize(fo))

# sujeito a
mdl.add(mdl.sum([x[i][j] for j in places]) == 1 for i in clients)
mdl.add(x[i][j]-y[j] <= 0 for i in clients for j in places)
mdl.add(mdl.sum(y[j] for j in places) == p)

print("\nImprimindo solução....")
msol = mdl.solve(TimeLimit=60, Workers=1)

if msol:
    print(msol.print_solution())
    print("Status: " + msol.get_solve_status())
else:
    print("Nenhuma solução encontrada")


Imprimindo solução....
-------------------------------------------------------------------------------
Model constraints: 46, variables: integer: 48, interval: 0, sequence: 0
Solve status: Optimal
Search status: SearchCompleted, stop cause: SearchHasNotBeenStopped
Solve time: 0.1 sec
-------------------------------------------------------------------------------
Objective values: (41.67,), bounds: (41.6658,), gaps: (0.0001,)
X[0][0] 25.0=0
X[0][1] 6.67=1
X[0][2] 6.67=1
X[0][3] 25.0=0
X[0][4] 18.33=0
X[1][0] 21.67=0
X[1][1] 16.67=0
X[1][2] 16.67=0
X[1][3] 8.33=1
X[1][4] 21.67=0
X[2][0] 21.67=0
X[2][1] 11.67=0
X[2][2] 15.0=0
X[2][3] 11.67=0
X[2][4] 13.33=0
X[3][0] 13.33=1
X[3][1] 20.0=0
X[3][2] 13.33=0
X[3][3] 23.33=0
X[3][4] 6.67=1
X[4][0] 8.33=0
X[4][1] 15.0=0
X[4][2] 13.33=0
X[4][3] 16.67=0
X[4][4] 21.67=0
X[5][0] 18.33=0
X[5][1] 15.0=0
X[5][2] 16.67=0
X[5][3] 21.67=0
X[5][4] 23.33=0
X[6][0] 8.33=0
X[6][1] 8.33=0
X[6][2] 21.67=0
X[6][3] 20.0=0
X[6][4] 18.33=0
X[7][0] 21.67=0
X[7][1] 

### créditos
Este notebook foi feito com auxílio dos vídeos:

[1 - Network Optimization | P-Median Problem](https://www.youtube.com/watch?v=GytzJiuYUe4)

[Modelando o Problema de p medianas usando o CPLEX (aula)](https://www.youtube.com/watch?v=Fc9SsoPRDxo)


### Referências
* SENNE, Edson Luiz França; LORENA, Luiz Antonio Nogueira. Abordagens complementares para problemas de p-medianas. Production, v. 13, n. 3, p. 78-87, 2003.